# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 30 2022 2:36PM,240880,12,U0952,"Uniderm USA, Inc",Released
1,Jun 30 2022 2:29PM,240879,10,0085817266,ISDIN Corporation,Released
2,Jun 30 2022 2:29PM,240879,10,0085817267,ISDIN Corporation,Released
3,Jun 30 2022 2:29PM,240879,10,0085817268,ISDIN Corporation,Released
4,Jun 30 2022 2:29PM,240879,10,0085817308,ISDIN Corporation,Released
5,Jun 30 2022 2:29PM,240879,10,0085817319,ISDIN Corporation,Released
6,Jun 30 2022 2:29PM,240879,10,0085817363,ISDIN Corporation,Released
7,Jun 30 2022 2:29PM,240879,10,0085817392,ISDIN Corporation,Released
8,Jun 30 2022 2:29PM,240879,10,0085817394,ISDIN Corporation,Released
9,Jun 30 2022 2:29PM,240879,10,0085817404,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,240875,Released,3
28,240876,Released,7
29,240878,Released,1
30,240879,Released,14
31,240880,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240875,NaN,NaN,3.0
240876,NaN,NaN,7.0
240878,NaN,NaN,1.0
240879,NaN,NaN,14.0
240880,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240774,0.0,1.0,0.0
240781,0.0,0.0,1.0
240784,0.0,0.0,1.0
240786,0.0,0.0,1.0
240800,24.0,27.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240774,0,1,0
240781,0,0,1
240784,0,0,1
240786,0,0,1
240800,24,27,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240774,0,1,0
1,240781,0,0,1
2,240784,0,0,1
3,240786,0,0,1
4,240800,24,27,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240774,,1,
1,240781,,,1
2,240784,,,1
3,240786,,,1
4,240800,24,27,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 30 2022 2:36PM,240880,12,"Uniderm USA, Inc"
1,Jun 30 2022 2:29PM,240879,10,ISDIN Corporation
15,Jun 30 2022 2:26PM,240878,10,Cosmo International Fragrances
16,Jun 30 2022 2:22PM,240876,12,Hush Hush
23,Jun 30 2022 2:17PM,240875,10,Emerginnova
26,Jun 30 2022 2:16PM,240874,10,Emerginnova
28,Jun 30 2022 2:02PM,240873,16,Sartorius Stedim North America
29,Jun 30 2022 1:55PM,240872,19,"AdvaGen Pharma, Ltd"
30,Jun 30 2022 1:21PM,240870,20,Alumier Labs Inc.
38,Jun 30 2022 12:31PM,240861,16,Sartorius Stedim Filters Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 30 2022 2:36PM,240880,12,"Uniderm USA, Inc",,,1
1,Jun 30 2022 2:29PM,240879,10,ISDIN Corporation,,,14
2,Jun 30 2022 2:26PM,240878,10,Cosmo International Fragrances,,,1
3,Jun 30 2022 2:22PM,240876,12,Hush Hush,,,7
4,Jun 30 2022 2:17PM,240875,10,Emerginnova,,,3
5,Jun 30 2022 2:16PM,240874,10,Emerginnova,,,2
6,Jun 30 2022 2:02PM,240873,16,Sartorius Stedim North America,,,1
7,Jun 30 2022 1:55PM,240872,19,"AdvaGen Pharma, Ltd",,,1
8,Jun 30 2022 1:21PM,240870,20,Alumier Labs Inc.,,,8
9,Jun 30 2022 12:31PM,240861,16,Sartorius Stedim Filters Inc.,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 2:36PM,240880,12,"Uniderm USA, Inc",1,,
1,Jun 30 2022 2:29PM,240879,10,ISDIN Corporation,14,,
2,Jun 30 2022 2:26PM,240878,10,Cosmo International Fragrances,1,,
3,Jun 30 2022 2:22PM,240876,12,Hush Hush,7,,
4,Jun 30 2022 2:17PM,240875,10,Emerginnova,3,,
5,Jun 30 2022 2:16PM,240874,10,Emerginnova,2,,
6,Jun 30 2022 2:02PM,240873,16,Sartorius Stedim North America,1,,
7,Jun 30 2022 1:55PM,240872,19,"AdvaGen Pharma, Ltd",1,,
8,Jun 30 2022 1:21PM,240870,20,Alumier Labs Inc.,8,,
9,Jun 30 2022 12:31PM,240861,16,Sartorius Stedim Filters Inc.,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 2:36PM,240880,12,"Uniderm USA, Inc",1,,
1,Jun 30 2022 2:29PM,240879,10,ISDIN Corporation,14,,
2,Jun 30 2022 2:26PM,240878,10,Cosmo International Fragrances,1,,
3,Jun 30 2022 2:22PM,240876,12,Hush Hush,7,,
4,Jun 30 2022 2:17PM,240875,10,Emerginnova,3,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 2:36PM,240880,12,"Uniderm USA, Inc",1.0,NaN,NaN
1,Jun 30 2022 2:29PM,240879,10,ISDIN Corporation,14.0,NaN,NaN
2,Jun 30 2022 2:26PM,240878,10,Cosmo International Fragrances,1.0,NaN,NaN
3,Jun 30 2022 2:22PM,240876,12,Hush Hush,7.0,NaN,NaN
4,Jun 30 2022 2:17PM,240875,10,Emerginnova,3.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 2:36PM,240880,12,"Uniderm USA, Inc",1.0,0.0,0.0
1,Jun 30 2022 2:29PM,240879,10,ISDIN Corporation,14.0,0.0,0.0
2,Jun 30 2022 2:26PM,240878,10,Cosmo International Fragrances,1.0,0.0,0.0
3,Jun 30 2022 2:22PM,240876,12,Hush Hush,7.0,0.0,0.0
4,Jun 30 2022 2:17PM,240875,10,Emerginnova,3.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1445196,21.0,12.0,0.0
12,481756,8.0,0.0,0.0
15,240847,0.0,29.0,0.0
16,963357,4.0,0.0,0.0
19,963307,6.0,29.0,24.0
20,1204240,25.0,16.0,0.0
21,963125,3.0,1.0,0.0
22,240846,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1445196,21.0,12.0,0.0
1,12,481756,8.0,0.0,0.0
2,15,240847,0.0,29.0,0.0
3,16,963357,4.0,0.0,0.0
4,19,963307,6.0,29.0,24.0
5,20,1204240,25.0,16.0,0.0
6,21,963125,3.0,1.0,0.0
7,22,240846,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,21.0,12.0,0.0
1,12,8.0,0.0,0.0
2,15,0.0,29.0,0.0
3,16,4.0,0.0,0.0
4,19,6.0,29.0,24.0
5,20,25.0,16.0,0.0
6,21,3.0,1.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,21.0
1,12,Released,8.0
2,15,Released,0.0
3,16,Released,4.0
4,19,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
Executing,12.0,0.0,29.0,0.0,29.0,16.0,1.0,0.0
Released,21.0,8.0,0.0,4.0,6.0,25.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
1,Executing,12.0,0.0,29.0,0.0,29.0,16.0,1.0,0.0
2,Released,21.0,8.0,0.0,4.0,6.0,25.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
1,Executing,12.0,0.0,29.0,0.0,29.0,16.0,1.0,0.0
2,Released,21.0,8.0,0.0,4.0,6.0,25.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()